In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")
# Specify the path to your existing CSV file
csv_file_path = 'amazon_validation4.csv'

# Read ASINs from 'ASINS.csv'
asins_file_path = 'ASINs_validation.csv'
with open(asins_file_path, 'r') as asins_file:
    # Assuming ASINs are in the first column of the CSV file
    #asins_list = [row[0] for row in csv.reader(asins_file)]
    # Assuming ASINs are in the first column of the CSV file
    asins_list = [row[0] for row in csv.reader(asins_file) if row]

# Create a list to store failed ASINs
failed_asins_list = []

# Open the CSV file in append mode
with open(csv_file_path, 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)

    # Check if the file is empty and write headers if needed
    if f.tell() == 0:
        # writer.writerow(['ASIN', 'Title','products_overview','bullet_points','product_information','product_description'])
        writer.writerow(['ASIN', 'Title','A_plus_content','buybox_owner','Price','products_overview','bullet_point','product_information','product_description','Review_check','image_block','Bread_crumb'])

    # Iterate through each ASIN and fetch data simultaneously
    for asin in asins_list:
        retry_count = 2  # Number of retries
        while retry_count > 0:
            # Create the URL for the current ASIN
            base_url = "https://www.amazon.com/dp/"
            url = base_url + asin

            # Make a request to the Amazon product page
            headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Ch": "ect,rtt,downlink,device-memory,sec-ch-device-memory,viewport-width,sec-ch-viewport-width,dpr,sec-ch-dpr,sec-ch-ua-platform-version",
    "Accept-Ch-Lifetime": "86400",
    "Alt-Svc": "h3=\":443\"; ma=86400",
    "Accept-Encoding": "gzip, deflate,br",
    "authority": "www.amazon.com",
    "method": "POST",
    "path": "/vap/ew/componentbuilder",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1",
    "Cookie": """session-id=143-0791319-6859503; ubid-main=131-7578819-3400663; sid="xCcOs8TFqzbkMnM2n/ZHWQ==|ztgsA06HyNFh1GoT8HJ3XKLJ3WVrtI/s7+DpE2EFXKY="; AMCV_A7493BC75245ACD20A490D4D%40AdobeOrg=1585540135%7CMCIDTS%7C20104%7CMCMID%7C08021445027975837283202940264552163542%7CMCAAMLH-1737622589%7C12%7CMCAAMB-1737622589%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1737024989s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.0; i18n-prefs=USD; s_pers=%20s_fid%3D6A473DFA58024449-0DFF97CD5D3B973D%7C1895131206117%3B%20s_dl%3D1%7C1737366606134%3B%20s_ev15%3D%255B%255B%2527NSGoogle%2527%252C%25271736514899737%2527%255D%252C%255B%2527Google%2527%252C%25271736921267466%2527%255D%252C%255B%2527NSGoogle%2527%252C%25271737364806248%2527%255D%255D%7C1895131206248%3B; x-amz-captcha-1=1737619376181776; x-amz-captcha-2=5kXbewcAyCDs61MhUfVCTQ==; lc-main=en_US; sst-main=Sst1|PQFq9AlpZ6DTM4JuapLInl-yDHaL47M9vZVQ05pKx3lYOrADwh4422fy1x8fCsgHGIx5bw3JK1Z8270vtABKln-uy5gQFgSUV96FzkkYly2L37fCncpGzulYRvUo2_4tku9OVz-x41-KoP0Vl-1TiAGbM-x9ER9Tn6hxko0a6UlVbAgPKK1QBggxpm0xDKWDsP-mmpz5LsIYDOxtxYqNgu47QuZf3k86pNTQfxSjxthMJAgTG95DpUKWvdKgvv-st73tnr7cJQz4GvgrKb_OFtnNJe2xrZ5BPNrUp_9jmBXsqBNUBxkv9c1ToC8UvwIrJFtDsivGcen-9O7PZBywKOVYgixq9fCztafPTQzeI94v7_0; ph_phc_tGCcl9SINhy3N0zy98fdlWrc1ppQ67KJ8pZMzVZOECH_posthog=%7B%22distinct_id%22%3A%22amzn1.account.ocid.A3E4QLZBKGG966%22%2C%22%24sesid%22%3A%5B1737956488189%2C%220194a647-6daf-7daf-abb2-2605dd4da93c%22%2C1737956486575%5D%2C%22%24epp%22%3Atrue%7D; at-main=Atza|IwEBIIdzVw3_HddPJhwR3-6T9tTMJMhpf1FPVWp6-ZdYkOZvsKF4StRXqeCbTvZ2vQPQ6AQUoxEccZgFWwaK_MKyBkguLHkhoE0_63tZz3AR88eojZLIIIcaybCk7NxgN0G_jYFVNoz8Fueby_jc73WhrHD7PwGiEhwHgfutEW9BoYQAQoc5k0dawUWxIEGv6s3WgAqiJLfFlUPFYra7j78oSmonasm3uVVpQ__sM9OWSe7M4yySiWffMjmo4X_d-JtXNZ4_w-jrGr6OUlcyW1QzcgDoWPyc_tpbI-SKsETlNn3Mpg; sess-at-main="fLmFy54xXV/FEO6o6CKi/azN+NYyzE0MqF4Ex4hJubs="; session-id-time=2082787201l; session-token=wI8xZCnfwh3g1n4ehD27eYFB/G+1CSVCdmGgN/FO4AkB6nrTArvmZILftsdm7OFv701jyCEtjgqyBb/AB2JOglCKVTjK22cSQGBscQybNKWb6h8kv84x/peC/jg927XTiY0GmMYpuTvpKJe9r/9pJvi3jxp5t03lS6NBzkZlThtxI6I6U4rEvuoOTRnQ8Njo5A/oy5PIJqTaP4EuPQRwr0cSsmXqeEeezpWbvNel00OfTLAUFHquEE2/xk0QCxpfkmGQ0T6Zrsnf/vfrOD6LyiJ+LaXVL56A7c0OJAYeQL/Qxf4SwPRNIHRvWeZcVw8szDyoQhK2HhzVJ5hRMzt6OjDx4bVusUUtPDvfwvXO3SymZE5rMJuLJNDFjh0x5Yi2; x-main="cYsuJw1zamq1PDe59Vft3SrJkoFVcX2wZE12LRNTkzcf?flHZSnoxP5QFtkVveTs"; csm-hit=tb:s-Z8KGK8V4ZSNFK925PRAF|1739707632377&t:1739707638012&adb:adblk_no"""
}
            try:
                page = requests.get(url, headers=headers)
                time.sleep(0.8)  # Introduce a delay between requests
            except Exception as e:
                print(f"Error fetching data for ASIN {asin}: {e}")
                # Append the ASIN to the failed_asins_list
                failed_asins_list.append({'ASIN': asin, 'Reason': str(e)})
                continue  # Retry if there is an error

            # Parse the HTML content
            soup1 = BeautifulSoup(page.content, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            # Extract data from the page
            title = soup2.find(id='productTitle').get_text().strip() if soup2.find(id='productTitle') else 'N/A'
            aplus_element = soup2.find(id='aplus')
            A_plus_content = aplus_element.get_text().strip() if aplus_element else 'N/A'
            Bread_crumb = soup2.find(id='wayfinding-breadcrumbs_feature_div').get_text().strip() if soup2.find(
                id='wayfinding-breadcrumbs_feature_div') else 'N/A'
            buybox_owner = soup2.find(id= "offer-display-features") if soup2.find(id= "offer-display-features") else 'N/A'
            price = 'N/A'
            unavailable_count = soup2.get_text().count("Currently unavailable")            

            # Check if the item is unavailable
            if unavailable_count >= 2:
                price = 'N/A'
                print(f"ASIN {asin} is unavailable. Setting price to 'N/A'.")
            else:
                # Attempt to find price in various sections
                if price == 'N/A':
                    print("Attempting to find price in multiple sections.")
                    
                    # List of potential sections with price spans
                    potential_ids = [
                        'corePriceDisplay_desktop_feature_div',  # Main target
                        'priceblock_ourprice',                  # Fallback for list price
                        'priceblock_dealprice',                 # For deals
                        'corePrice_feature_div',
                        'aok-offscreen'
                                                    # Another possible section
                    ]
                    
                    price_section = None
                    for section_id in potential_ids:
                        price_section = soup2.find('div', id=section_id)
                        if price_section:
                            break
                    
                    if not price_section:
                        # Check for price in table structure with 'Bundle Price' and 'Bundle List Price'
                        table_section = soup2.find('table', class_='a-lineitem a-align-top')
                        if table_section:
                            # Extract 'Bundle List Price'
                            bundle_list_price_span = table_section.find('span', class_='a-price a-text-price a-size-base')
                            if bundle_list_price_span:
                                bundle_list_price = bundle_list_price_span.find('span', class_='a-offscreen')
                                if bundle_list_price:
                                    price = bundle_list_price.get_text().strip()
                                    print(f"Bundle List Price found: {price}")
                            
                            # Extract 'Bundle Price'
                            if price == 'N/A':  # Only search if Bundle List Price is not found
                                bundle_price_span = table_section.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay')
                                if bundle_price_span:
                                    bundle_price = bundle_price_span.find('span', class_='a-offscreen')
                                    if bundle_price:
                                        price = bundle_price.get_text().strip()
                                        print(f"Bundle Price found: {price}")
                    
                    # Broader search for any price element (fallback)
                    if price == 'N/A':
                        print("Attempting to find price in any span with 'a-price' class.")
                        price_spans = soup2.find_all('span', class_='a-price')
                        for price_span in price_spans:
                            offscreen_price = price_span.find('span', class_='a-offscreen')
                            if offscreen_price and offscreen_price.get_text().strip():
                                price = offscreen_price.get_text().strip()
                                print(f"Fallback price found: {price}")
                                break  # Exit loop once a valid price is found            

                    if price_section:
                        # Search for the price in the section
                        price_block = price_section.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
                        if price_block:
                            price_symbol = price_block.find('span', class_='a-price-symbol')
                            price_whole = price_block.find('span', class_='a-price-whole')
                            price_fraction = price_block.find('span', class_='a-price-fraction')            

                            if price_symbol and price_whole and price_fraction:
                                price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
                                print(f"Price found: {price}")
                            else:
                                print("Price components not found in the specified structure.")
                        else:
                            print("Price block not found in the identified section.")
                    else:
                        print("No relevant price section found.")
            products_overview = soup2.find('div', {'data-csa-c-content-id':'productOverview'}) 
            #product_overview = soup2.find(id='productOverview_feature_div').get_text().strip() if soup2.find(id='productOverview_feature_div') else 'N/A'
            bullet_point_element = soup2.find(id='productFactsDesktopExpander') or soup2.find(id='feature-bullets')
            bullet_point = bullet_point_element.get_text().strip() if bullet_point_element else 'N/A'
            product_information = soup2.find(id='prodDetails').get_text().strip() if soup2.find(id='prodDetails') else 'N/A'
            product_description = soup2.find(id='productDescription').get_text().strip() if soup2.find(id='productDescription') else 'N/A'
            Review_check = soup2.find(id='reviewsMedley').get_text().strip() if soup2.find(
                id='reviewsMedley') else 'N/A'
                        # Locate the #imageBlock_feature_div section
            image_block_feature_div = soup2.find("div", id="imageBlock_feature_div")            

             # Locate the desired script within the #imageBlock_feature_div
            image_block_script = (
                    image_block_feature_div.find("script", text=lambda t: t and "P.when('A').register(\"ImageBlockATF\"" in t)
                    if image_block_feature_div
                    else None
            )
            image_block = image_block_script.string.strip() if image_block_script else 'N/A'
            # image_block = image_block_script.text.strip() if image_block_script else 'N/A'
            # imgs_str = img_div.img.get('data-a-dynamic-image')  # a string in Json format
            # Reviews_overall = soup2.find(id='customerReviews').get_text().strip() if soup2.find(
            #    id='customerReviews') else 'N/A'
            # Insights_reviews = soup2.find(id='cr-product-insights-cards').get_text().strip() if soup2.find(
            #    id='cr-product-insights-cards') else 'N/A'
            # Twister_data = soup2.find(id='twister').get_text().strip() if soup2.find(
            #    id='twister') else 'N/A'
                

            # Check if title or ratings is 'N/A' and retry if needed
            if title == 'N/A':
                retry_count -= 1
                print(f"Retrying for ASIN {asin}. Retries left: {retry_count}")
            else:
                break  # Break out of the retry loop if successful

        # If all retries failed, print an error message
        if retry_count == 0:
            print(f"Failed to fetch data for ASIN {asin} after multiple retries.")
            # Append the ASIN to the failed_asins_list
            failed_asins_list.append({'ASIN': asin, 'Reason': 'Max retries reached'})
            continue

        # Write the data to the CSV file
        writer.writerow([asin, title,A_plus_content,buybox_owner,price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb])
        # writer.writerow([asin, title, price])

# Convert the failed_asins_list to a DataFrame
failed_asins_df = pd.DataFrame(failed_asins_list)

# Save the failed ASINs to a separate CSV file
failed_asins_df.to_csv('failed_asins-final.csv', index=False)

In [1]:
import pandas as pd
validation_df1=pd.read_csv(r"C:\Users\sudha\Amazon validation script\amazon_validation7.csv")

pd.set_option("display.max_columns", None)
validation_df1


,ASIN,Title,A_plus_content,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B00DOOFTAU,Restaurantware 4 Ounce Mini Dessert Cups 100 S...,From the brand\n \n\n\n\n\n\n\n\n\n\n...,"<div class=""a-section a-spacing-base offer-dis...",$69\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Infuse a touch of elegance into your serving s...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
1,B00NJSYCMM,Restaurantware 9 x 9 Inch Plates With Glass Ho...,From the brand\n \n\n\n\n\n\n\n\n\n\n...,"<div class=""a-section a-spacing-base offer-dis...",$188\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Make it easy for guests to sip and snack with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2,B00DOOCR7I,"Restaurantware 7 Inch Bamboo Cones, 200 Compos...",From the brand\n \n\n\n\n\n\n\n\n\n\n...,"<div class=""a-section a-spacing-base offer-dis...",$83\n \n ..54,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,A rustic and unique alternative to serving sna...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B007Y1LJIU,Restaurantware 2 Ounce Dessert Dishes 100 Disp...,From the brand\n \n\n\n\n\n\n\n\n\n\n...,"<div class=""a-section a-spacing-base offer-dis...",$46\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Impress your guests with our 2-ounce Square Se...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B00DOOD7W2,Restaurantware 3 Ounce Appetizer Cups 200 Disp...,From the brand\n \n\n\n\n\n\n\n\n\n\n...,"<div class=""a-section a-spacing-base offer-dis...",$104\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Serve small portions or samples in style with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...
9583,B0D2P5HZLM,"Restaurantware Bio Tek 71 Ounce To Go Boxes, 4...",From the manufacturer\n \n\n\n\n\n\n\...,"<div class=""a-section a-spacing-base offer-dis...",$163\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0D2P5...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Upgrade your food packaging with our Bio Tek 7...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\n...
9584,B0D2P5DPQM,"Restaurantware Bio Tek 71 Ounce To Go Boxes, 4...",From the manufacturer\n \n\n\n\n\n\n\...,"<div class=""a-section a-spacing-base offer-dis...",$163\n \n ..79,"<div class=""celwidget"" data-csa-c-asin=""B0D2P5...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Upgrade your food packaging with our Bio Tek 7...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\n...
9585,B0D2P3WPPX,"Restaurantware Bio Tek 71 Ounce To Go Boxes, 1...",From the manufacturer\n \n\n\n\n\n\n\...,"<div class=""a-section a-spacing-base offer-dis...",$43\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0D2P3...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Effortlessly package your mouthwatering food i...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\n...
9586,B0D2P36X1P,"Restaurantware Bio Tek 47 Ounce To

In [4]:
import pandas as pd
validation_df3=pd.read_csv('amazon_validation4.csv')

pd.set_option("display.max_columns", None)
validation_df3

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B01MYXVYUF,"3.5"" Voga Melamine Chopstick and Spoon Rest, S...","<div class=""a-section a-spacing-base offer-dis...",$27\n \n ..36,"<div class=""celwidget"" data-csa-c-asin=""B01MYX...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1,B07Z4JHJZW,Bar Lux Black Plastic Condiment Caddy Napkin H...,"<div class=""a-section a-spacing-base offer-dis...",$41\n \n ..29,"<div class=""celwidget"" data-csa-c-asin=""B07Z4J...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2,B081BN3MXS,10 oz Square Gray Aluminum Baking Cup - Lids A...,"<div class=""a-section a-spacing-base offer-dis...",$186\n \n ..08,"<div class=""celwidget"" data-csa-c-asin=""B081BN...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B01B5KZ9CM,Rose Gold Baking Cups - Foil Coated Baking Cup...,"<div class=""a-section a-spacing-base offer-dis...",$42\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B01B5K...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B0CFVXHF92,Restaurantware Balloonify 16 Inch Number Ballo...,"<div class=""a-section a-spacing-base offer-dis...",$10\n \n ..88,"<div class=""celwidget"" data-csa-c-asin=""B0CFVX...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Toys & Games\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...
235,B0D35ZXTSK,Restaurantware - Coco Casa 5 Inch Handmade Ser...,"<div class=""a-section a-spacing-base offer-dis...",$219\n \n ..56,"<div class=""celwidget"" data-csa-c-asin=""B0D35Z...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
236,B0CSG3TKBX,Restaurantware Foil Lux 10.8 x 9 Inch Pop-Up F...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0CSG3...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Industrial & Scientific\n \n\n\...
237,B0D8QLRZ46,"Restaurantware-Bento Tek 57.5 Ounce Bento Box,...","<div class=""a-section a-spacing-base offer-dis...",$22\n \n ..25,"<div class=""celwidget"" data-csa-c-asin=""B0D8QL...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
238,B0DNK1L7JW,Restaurantware-Met Lux 43 Ounce Hinged Food St...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0DNK1...",About this item\n \n\n\n\n ...,NaN,NaN,Customer reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...


In [4]:
import pandas as pd
validation_df3=pd.read_csv('amazon_validation3.csv')

pd.set_option("display.max_columns", None)
validation_df3

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B0CN6WCPVB,Restaurantware Panificio 4.5 Ounce Baking Cups...,"<div class=""a-section a-spacing-base offer-dis...",$16\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0CN6W...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Add a touch of sophistication to your pastry p...,Customer reviews\n \n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1,B0CN6YH52F,Restaurantware Panificio 3.5 Ounce Baking Cups...,"<div class=""a-section a-spacing-base offer-dis...",$12\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0CN6Y...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Elevate your baking presentation with our Pani...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2,B0CN6WSH7M,"Panificio 3.5 Ounce Baking Cups, 50 Disposable...","<div class=""a-section a-spacing-base offer-dis...",$13\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0CN6W...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Raise your baking standards with our Panificio...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B0CN6XR22B,Restaurantware Panificio 3.9 Ounce Baking Cups...,"<div class=""a-section a-spacing-base offer-dis...",$16\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0CN6X...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Stand out at your party dessert presentation w...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B0CN6YJRW6,Restaurantware Panificio 4.5 Ounce Baking Cups...,NaN,$15.74,"<div class=""celwidget"" data-csa-c-asin=""B0CN6Y...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Give your dessert presentation a unique touch ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...
3808,B0DNK83Z6F,Restaurantware-Cater Tek 51 Ounce Serving Bowl...,"<div class=""a-section a-spacing-base offer-dis...",$111\n \n ..76,"<div class=""celwidget"" data-csa-c-asin=""B0DNK8...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Deliver exceptional catering experiences with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3809,B0DQLGYV67,Restaurantware-Sustain 20 Ounce Charcuterie Cu...,"<div class=""a-section a-spacing-base offer-dis...",$32\n \n ..79,"<div class=""celwidget"" data-csa-c-asin=""B0DQLG...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Elevate your take out presentation with our Su...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
3810,B0DQLF3PQW,Restaurantware-Sustain 16 Ounce Charcuterie Cu...,"<div class=""a-section a-spacing-base offer-dis...",$32\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0DQLF...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Elevate your food presentation with our Sustai...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
3811,B0DQLF912T,Restaurantware-Sustain 16 Ounce Charcuterie Cu...,"<div class=""a-section a-spacing-base offer-dis...",$31\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B0DQLF...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Elevate your food presentation with our Sustai...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...


In [1]:
import pandas as pd
validation_df2=pd.read_csv('amazon_validation3.csv')

pd.set_option("display.max_columns", None)
validation_df2

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B081BNC2D9,Round Clear Plastic Flat Lid - Fits 4 oz Alumi...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B081BN...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Keep sweets sealed in using our Round Clear Pl...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
1,B092W43FCJ,"RW Base Clear Industrial Tape - 2 mil - 3"" x 1...","<div class=""a-section a-spacing-base offer-dis...",$126\n \n ..10,"<div class=""celwidget"" data-csa-c-asin=""B092W4...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Easily pack your glassware in boxes with our R...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2,B09QLYDTPJ,Pastry Tek Silicone Mini Bundt Cake Baking Mol...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B09QLY...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Bake a variety of Bundt cakes for dessert stat...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B0C7RK1P7J,Pulp Safe No PFAS Added 26 Ounce Oval Salad Pl...,NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B0C7RK...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Impress your guests with your signature salads...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B082XWYV46,"Coppetta 12 oz Round Red Paper to Go Cup - 4"" ...","<div class=""a-section a-spacing-base offer-dis...",$75\n \n ..36,"<div class=""celwidget"" data-csa-c-asin=""B082XW...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Améliorez la façon dont vous servez les desser...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...
1179,B00DOOFUXQ,"Mini Dessert Tumbler, Dessert Cup, Square Plas...",NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,These ultra-versatile mini cups are great for ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Personal Care\n \n\n\n...
1180,B09SZ8N984,Restaurantware LIDS ONLY: Foil Lux Plastic Lid...,"<div class=""a-section a-spacing-base offer-dis...",$62\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B09SZ8...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Make transporting catering orders effortless u...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1181,B08NQ2RPL1,Met Lux Rectangle Clear Plastic 1/3 Size Cold ...,"<div class=""a-section a-spacing-base offer-dis...",$291\n \n ..19,"<div class=""celwidget"" data-csa-c-asin=""B08NQ2...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Store prepped ingredients in your commercial r...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
1182,B06XYK419C,Mini Casserole Pot with Lid - Stainless Steel ...,"<div class=""a-section a-spacing-base offer-dis...",$26\n \n ..53,"<div class=""celwidget"" data-csa-c-asin=""B06XYK...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,This Small Casserole Pots are perfect for serv...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...


In [2]:
import pandas as pd

# List of CSV files to combine
files = ['amazon_validation11.csv']

# Initialize an empty list to hold the DataFrames
dfs = []

# Loop over the list of files and read each one into a DataFrame
for filename in files:
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
validation_df = pd.concat(dfs, ignore_index=True)

# Write the combined DataFrame to a new CSV file
validation_df.to_csv('combined.csv', index=False)

print("CSV files combined successfully!")

CSV files combined successfully!


In [3]:
import pandas as pd
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
validation_df = pd.read_csv('combined.csv')
validation_df

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B00DOOFTAU,Restaurantware 4 Ounce Mini Dessert Cups 100 S...,"<div class=""a-section a-spacing-base offer-dis...",$69\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Infuse a touch of elegance into your serving s...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
1,B00NJSYCMM,Restaurantware 9 x 9 Inch Plates With Glass Ho...,"<div class=""a-section a-spacing-base offer-dis...",$188\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Make it easy for guests to sip and snack with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
2,B00DOOCR7I,"Restaurantware 7 Inch Bamboo Cones, 200 Compos...","<div class=""a-section a-spacing-base offer-dis...",$83\n \n ..54,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,NaN,A rustic and unique alternative to serving sna...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
3,B007Y1LJIU,Restaurantware 2 Ounce Dessert Dishes 100 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$46\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Impress your guests with our 2-ounce Square Se...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
4,B00DOOD7W2,Restaurantware 3 Ounce Appetizer Cups 200 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$113\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Serve small portions or samples in style with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...
11691,B0DT9RFTWY,"Restaurantware-RW Clean, 33 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$43\n \n ..41,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Efficiently tackle your kitchen waste with our...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
11692,B0DT9PRL11,"Restaurantware-RW Clean, 45 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$52\n \n ..35,"<div class=""celwidget"" data-csa-c-asin=""B0DT9P...",About this item\n \n\n\n\n ...,NaN,Make waste disposal easier than ever with our ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
11693,B0DT9RY8H9,"Restaurantware-RW Clean, 45 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$74\n \n ..78,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Simplify your kitchen waste disposal with our ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...
11694,B0DT9RQHK1,"Restaurantware-RW Clean, 55 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$61\n \n ..33,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Keep your outdoor events clean and organized w...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...


In [4]:
validation_df.product_description[1]

'Make it easy for guests to sip and snack with our Square Natural Bamboo Leaf Plates with Cup Holder. Equipped with a circular cutout to accommodate stemware, these bamboo plates enable guests to carry their plates and wine glasses in the same hand. Plus, these bamboo leaf plates are designed with a circular indent that acts as a convenient built-in coaster when the plates are laid flat. These appetizer plates with wine glass holders are made from bamboo and are strong enough to hold all kinds of food. Each cocktail plate with cup holder measures 9 inches long by 9 inches wide and the cup holder has a diameter of 1.8 inches. Shipped in a 50 count box.'

In [5]:
validation_df.products_overview[1]

'<div class="celwidget" data-csa-c-asin="B00NJSYCMM" data-csa-c-content-id="productOverview" data-csa-c-is-in-initial-active-row="false" data-csa-c-slot-id="productOverview_feature_div" data-csa-c-type="widget" data-feature-name="productOverview" id="productOverview_feature_div">\n<style>\n           .po-break-word {\n        word-break: break-word;\n    }\n    .po-truncate-attribute .a-truncate-cut{\n        word-break: break-word;\n    }\n    #po-tta-action {\n        padding: 0;\n    }\n          </style>\n<script type="text/javascript">\n           function logTechTermAssistMetric(nexusSchemaId, metricParameters) {\n        if (typeof window.csa !== \'undefined\') {\n            var ttaEvents = csa("Events", {producerId: "cetech-tta"});\n\n            metricParameters.schemaId = nexusSchemaId;\n\n            metricParameters.asin = \'B00NJSYCMM\';\n            metricParameters.deviceType = \'web\';\n            metricParameters.productType = \'DISHWARE_PLATE\';\n            metricP

In [6]:
validation_df.bullet_point[2]

'About this item\n           \n\n\n\n              SUSTAINABLE: Made from bamboo, this cone is a natural and nature-friendly alternative to paper and plastic serving cones.\n             \n\n\n\n              DURABLE: Designed to hold small food servings like chicken tenders, fries, fruit, vegetables, and other bite-sized morsels, these cones are sturdy and built to last. Each measures 3 inches in diameter and 7 inches in height.\n             \n\n\n\n              A CREATIVE WAY TO SERVE: The unique and rustic design allow for a more fun and creative way to serve up some of your tastiest bite-sized fare.\n             \n\n\n\n              VERSATILE: Ideal for food trucks, outdoor catered events, wedding receptions, cafes, or other establishments looking for a food-serving alternative to traditional bowls or plates.\n             \n\n\n\n              SOLD IN BULK: These cones are shipped in bulk in a 200 count box.\n             \n\n\n\n\n\n\n\n             ›\n            \n\n       

In [7]:
validation_df.product_information[0]

'Product information\n           \n\n\n\n\n\n\n\n                  Bowl Material\n                 \n\n                  Plastic\n                 \n\n\n\n                  Number of Pieces\n                 \n\n                  100\n                 \n\n\n\n                  Brand\n                 \n\n                  Restaurantware\n                 \n\n\n\n                  Color\n                 \n\n                  White\n                 \n\n\n\n                  Style\n                 \n\n                  100\n                 \n\n\n\n                  Special Feature\n                 \n\n                  Disposable\n                 \n\n\n\n                  Shape\n                 \n\n                  Square\n                 \n\n\n\n                  Capacity\n                 \n\n                  4 ounces\n                 \n\n\n\n                  Specific Uses For Product\n                 \n\n                  Cold Drinks\n                 \n\n\n\n             

# Cleaning bullet points

In [8]:
import pandas as pd

# Define the function to process bullet points, excluding specific phrases
def extract_bullet_points(text):
    # Define the list of terms to exclude from bullet points
    exclusions = [
        'About this item', 'Nullify', '›', 'See more product details', 'Show more', 'Product details', 'Origin', 
        'Imported', 'Country of Origin', 'China', 'Fabric type', 'Non-woven', 'Care instructions', 'Hand Wash Only', 
        'Made in the USA and Imported', 'Fit type', 'Stretch', 'Closure type', 'Elastic', 'Pull-On', 'Tie Strap', 
        'Solid', 'Pattern', 'Nonwoven', 'Hybrid Plastic', 'Hand Wash', 'Polyester', 'Machine Wash', 'Snap','Eco-friendly', 'Wool',
        '100% Polyester', 'Polyeter','Paper','Kraft Paper','Kraft,Straw','Plastic','Vinyl','Spandex','Microfiber','Nitrile','Pva Microfiber','Upscale and Stylish!','nonwoven','Straw','Leather','Micro Fibre',
    ]
    
    # Process the text to extract and clean bullet points
    if pd.notna(text):
        # Split the text by newlines and strip any leading/trailing spaces
        lines = text.split('\n')
        bullet_points = [line.strip() for line in lines if line.strip()]
        
        # Exclude the first element, if necessary
        if len(bullet_points) > 1:
            bullet_points = bullet_points[1:]  # Exclude the first bullet point
            
        # Filter out any lines containing excluded phrases
        cleaned_bullet_points = [
            sentence for sentence in bullet_points 
            if sentence not in exclusions and not any(exclude in sentence for exclude in exclusions)
        ]
        
        return cleaned_bullet_points
    else:
        return []

# Apply the function to the DataFrame column
validation_df['Cleaned_Bullet_Points'] = validation_df['bullet_point'].apply(extract_bullet_points)

# Display the result
print(validation_df['Cleaned_Bullet_Points'])


0        [WHITE SOPHISTICATION: Enhance your events wit...
1        [PREMIUM QUALITY: These bamboo plates are made...
2        [SUSTAINABLE: Made from bamboo, this cone is a...
3        [ENHANCE YOUR PRESENTATION: Infuse elegance to...
4        [SERVE SAVORY SNACKS: Perfect for small portio...
                               ...                        
11691    [MEDIUM-DUTY DESIGN: Crafted from LDPE plastic...
11692    [LIGHT-DUTY DESIGN: Crafted from LDPE plastic ...
11693    [HEAVY-DUTY DESIGN: Crafted from LDPE plastic ...
11694    [LIGHT-DUTY DESIGN: Crafted from LDPE plastic ...
11695    [HEAVY-DUTY DESIGN: Crafted from LDPE plastic ...
Name: Cleaned_Bullet_Points, Length: 11696, dtype: object


In [9]:
# Identify the length of each row in the 'Cleaned_Bullet_Points' column
validation_df['Length_of_Cleaned_Bullet_Points'] = validation_df['Cleaned_Bullet_Points'].apply(len)
validation_df

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb,Cleaned_Bullet_Points,Length_of_Cleaned_Bullet_Points
0,B00DOOFTAU,Restaurantware 4 Ounce Mini Dessert Cups 100 S...,"<div class=""a-section a-spacing-base offer-dis...",$69\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Infuse a touch of elegance into your serving s...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...,[WHITE SOPHISTICATION: Enhance your events wit...,5
1,B00NJSYCMM,Restaurantware 9 x 9 Inch Plates With Glass Ho...,"<div class=""a-section a-spacing-base offer-dis...",$188\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Make it easy for guests to sip and snack with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...,[PREMIUM QUALITY: These bamboo plates are made...,5
2,B00DOOCR7I,"Restaurantware 7 Inch Bamboo Cones, 200 Compos...","<div class=""a-section a-spacing-base offer-dis...",$83\n \n ..54,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,NaN,A rustic and unique alternative to serving sna...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...,"[SUSTAINABLE: Made from bamboo, this cone is a...",5
3,B007Y1LJIU,Restaurantware 2 Ounce Dessert Dishes 100 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$46\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Impress your guests with our 2-ounce Square Se...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...,[ENHANCE YOUR PRESENTATION: Infuse elegance to...,5
4,B00DOOD7W2,Restaurantware 3 Ounce Appetizer Cups 200 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$113\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Serve small portions or samples in style with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Home & Kitchen\n \n\n\n\n\n ...,[SERVE SAVORY SNACKS: Perfect for small portio...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,B0DT9RFTWY,"Restaurantware-RW Clean, 33 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$43\n \n ..41,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Efficiently tackle your kitchen waste with our...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...,[MEDIUM-DUTY DESIGN: Crafted from LDPE plastic...,5
11692,B0DT9PRL11,"Restaurantware-RW Clean, 45 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$52\n \n ..35,"<div class=""celwidget"" data-csa-c-asin=""B0DT9P...",About this item\n \n\n\n\n ...,NaN,Make waste disposal easier than ever with our ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...,[LIGHT-DUTY DESIGN: Crafted from LDPE plastic ...,5
11693,B0DT9RY8H9,"Restaurantware-RW Clean, 45 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$74\n \n ..78,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Simplify your kitchen waste disposal with our ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Health & Household\n \n\n\n\n\n ...,[HEAVY-DUTY DESIGN: Crafted from LDPE plastic ...,5
11694,B0DT9RQHK1,"Restaurantware-RW Clean, 55 Gallon Trash 

In [10]:
validation_df.Cleaned_Bullet_Points[34]

['SUSTAINABLE PINEWOOD CONSTRUCTION: Made from bamboo, these food boats are ecologically conscious, serving as a great alternative to traditional plates. Reduce your carbon footprint!',
 'ATTRACTS THE ATTENTION OF GUESTS: Add rustic flair to celebrations! Featuring a neutral finish and designs, these stylish food trays complement the look of entree and dessert presentations.',
 'AVOIDS MESSY SPILLS: Expertly built with a durable base and sturdy walls, these serving boats are impervious to grease neatly contain hefty foods without bending or breaking. Easily avoid messes!',
 'GREAT FOR SPACE-SAVING STORAGE: Designed to be conveniently stackable, these disposable food trays are ideal for maximizing storage space in your kitchen. Keep your prep area organized!',
 '200 PINEWOOD BOATS PER ORDER: Each pinewood food tray measures 2.3 inches long by 1.5 inches wide at the base, and 4.5 inches long by 2.5 inches wide at the top. Each order includes 200 boats.']

In [11]:
# Splitting the bullet points into separate columns
validation_df[['bullet_point1', 'bullet_point2', 'bullet_point3', 'bullet_point4', 'bullet_point5','bullet_point6','bullet_point7','bullet_point8','bullet_point9','bullet_point10']] = pd.DataFrame(validation_df['Cleaned_Bullet_Points'].apply(lambda x: x[:10] if len(x) >= 10 else x + [None] * (10 - len(x))).tolist())
validation_df

,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,...,bullet_point1,bullet_point2,bullet_point3,bullet_point4,bullet_point5,bullet_point6,bullet_point7,bullet_point8,bullet_point9,bullet_point10
0,B00DOOFTAU,Restaurantware 4 Ounce Mini Dessert Cups 100 S...,"<div class=""a-section a-spacing-base offer-dis...",$69\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Infuse a touch of elegance into your serving s...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",...,WHITE SOPHISTICATION: Enhance your events with...,SMOOTH & DURABLE: Constructed from premium pla...,SPACE-EFFICIENT DESIGN: With a uniquely square...,"STREAMLINED CLEANUP: Designed for single-use, ...",100 QUADRATO CUPS: Measuring 2 in length by 2....,None,None,None,None,None
1,B00NJSYCMM,Restaurantware 9 x 9 Inch Plates With Glass Ho...,"<div class=""a-section a-spacing-base offer-dis...",$188\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00NJS...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Make it easy for guests to sip and snack with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",...,PREMIUM QUALITY: These bamboo plates are made ...,"ECOLOGICALLY-CONSCIOUS: Made up of bamboo, the...",BUILT-IN COASTER: These leaf plates are design...,CONVENIENT CLEANUP: These round bamboo plates ...,50 SQUARE DINNER PLATES PER ORDER: Measuring 9...,None,None,None,None,None
2,B00DOOCR7I,"Restaurantware 7 Inch Bamboo Cones, 200 Compos...","<div class=""a-section a-spacing-base offer-dis...",$83\n \n ..54,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,NaN,A rustic and unique alternative to serving sna...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",...,"SUSTAINABLE: Made from bamboo, this cone is a ...",DURABLE: Designed to hold small food servings ...,A CREATIVE WAY TO SERVE: The unique and rustic...,"VERSATILE: Ideal for food trucks, outdoor cate...",SOLD IN BULK: These cones are shipped in bulk ...,None,None,None,None,None
3,B007Y1LJIU,Restaurantware 2 Ounce Dessert Dishes 100 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$46\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B007Y1...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Impress your guests with our 2-ounce Square Se...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",...,ENHANCE YOUR PRESENTATION: Infuse elegance to ...,FOR ALL MINI-TREATS: These mini dessert cups a...,"STURDY & DURABLE: Fabricated from PS plastic, ...",STREAMLINE CLEANUP: These disposable mini Gira...,100 SEAGREEN DESSERT DISHES: Containing up to ...,None,None,None,None,None
4,B00DOOD7W2,Restaurantware 3 Ounce Appetizer Cups 200 Disp...,"<div class=""a-section a-spacing-base offer-dis...",$113\n \n ...,"<div class=""celwidget"" data-csa-c-asin=""B00DOO...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Serve small portions or samples in style with ...,Customer reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",...,SERVE SAVORY SNACKS: Perfect for small portion...,ENHANCE RUSTIC AESTHETICS: Made from sustainab...,FOR ALL KINDS OF TREATS: These disposable medi...,ECOLOGICALLY CONSCIOUS: With a concern for the...,200 MEDIUM CUPS PER ORDER: Each pinewood cup m...,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,B0DT9RFTWY,"Restaurantware-RW Clean, 33 Gallon Trash Bags,...","<div class=""a-section a-spacing-base offer-dis...",$43\n \n ..41,"<div class=""celwidget"" data-csa-c-asin=""B0DT9R...",About this item\n \n\n\n\n ...,NaN,Efficiently tackle your kitchen waste with our...,Customer reviews\n \n\n\n\n...,"P.when('

In [12]:
validation_df.product_description[1572]

"Instantly upgrade your food presentation with our Met Lux Small Individual Casserole Pots with Lids. Designed for long-lasting use, these 9 ounce stainless steel casserole pots are great break-resistant options for your bistro, restaurant, catering event, and more. Sold with a coordinating lid, these multi-purpose mini casserole pots can be used for individual servings of savory butternut squash soup at your restaurant or for sweet apple crumble pie at your next catering event. Equipped with built-in handles, these casserole dishes with lids measure 5.8 inches long by 3 inches wide and ship in a 1 count box.\n              \n\n\n\n               Product Specifications\n              \n\n\n\n\n\n                 Product Material: Stainless Steel\n                \n\n\n\n\n\n                 Product Color: Silver\n                \n\n\n\n\n\n                 Pack Size: 1 count\n                \n\n\n\n\n\n                 Product Dimensions: 5.8''L x 3.0''W x 3.8''H\n                \n\

In [13]:
validation_df.product_description[39]

"Give your handcrafted cocktails and bite-sized appetizers a much-needed punch of pizzazz with these Red Bamboo Braided Picks. Featuring multi-hued braided detailing and bright red beaded accents, these drink picks are perfect for skewering lemon peels and orange wedges to create amazing cocktail decorations. With their easy-to-hold braided handle, these bamboo picks are excellent for serving bite-sized hors d'oeuvres like teriyaki chicken bites and bacon-wrapped dates. These braided bamboo picks are suitable for picking hot and cold foods. Measuring 4 inches long, these red picks are ideal for large catered events. Sold in a 1,000 count box."

In [14]:
import pandas as pd
import numpy as np

def process_product_description(row):
    product_description = row['product_description']
    
    # Ensure product_description is a string
    if isinstance(product_description, str):
        # Split the text by newline character
        parts = product_description.split('\n', 1)
        
        # Extract product_description
        if len(parts) > 0:
            row['product_description'] = parts[0].strip()
        else:
            row['product_description'] = None
        
        # Extract product_specifications if exists
        if len(parts) > 1:
            product_specifications = parts[1].strip()
            # Remove "Product Specifications" from product_specifications
            product_specifications = product_specifications.replace("Product Specifications", "").strip()
            # Remove newline characters
            product_specifications = product_specifications.replace('\n', '')
            row['product_specifications'] = product_specifications
        else:
            row['product_specifications'] = None
    else:
        row['product_description'] = None
        row['product_specifications'] = None
    
    return row

# Apply the processing function to the validation_df.product_description column
validation_df = validation_df.apply(process_product_description, axis=1)
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
validation_df.head


<bound method NDFrame.head of              ASIN                                              Title  \
0      B00DOOFTAU  Restaurantware 4 Ounce Mini Dessert Cups 100 S...   
1      B00NJSYCMM  Restaurantware 9 x 9 Inch Plates With Glass Ho...   
2      B00DOOCR7I  Restaurantware 7 Inch Bamboo Cones, 200 Compos...   
3      B007Y1LJIU  Restaurantware 2 Ounce Dessert Dishes 100 Disp...   
4      B00DOOD7W2  Restaurantware 3 Ounce Appetizer Cups 200 Disp...   
...           ...                                                ...   
11691  B0DT9RFTWY  Restaurantware-RW Clean, 33 Gallon Trash Bags,...   
11692  B0DT9PRL11  Restaurantware-RW Clean, 45 Gallon Trash Bags,...   
11693  B0DT9RY8H9  Restaurantware-RW Clean, 45 Gallon Trash Bags,...   
11694  B0DT9RQHK1  Restaurantware-RW Clean, 55 Gallon Trash Bags,...   
11695  B0DT9TBVCP  Restaurantware-RW Clean, 55 Gallon Trash Bags,...   

                                            buybox_owner  \
0      <div class="a-section a-spacing-base o

In [15]:
validation_df.product_specifications[1572]

"Product Material: Stainless Steel                                 Product Color: Silver                                 Pack Size: 1 count                                 Product Dimensions: 5.8''L x 3.0''W x 3.8''H                                 Product SKU: RWT0189"

In [16]:
import pandas as pd
import re

# Function to extract key-value pairs and create new columns
def extract_specifications(row):
    if pd.notnull(row['product_specifications']):
        # Use regex to split into key-value pairs
        specifications = re.split(r'\s{2,}', row['product_specifications'].strip())
        
        for spec in specifications:
            if ':' in spec:
                key, value = spec.split(':', 1)
                key = key.strip()
                value = value.strip()
                row[key] = value
    
    return row

# Apply the extraction function to the DataFrame
validation_df = validation_df.apply(extract_specifications, axis=1)

# Display the processed DataFrame
validation_df.head


<bound method NDFrame.head of              ASIN                                        Bread_crumb  \
0      B00DOOFTAU  Health & Household\n               \n\n\n\n\n ...   
1      B00NJSYCMM  Home & Kitchen\n               \n\n\n\n\n     ...   
2      B00DOOCR7I  Home & Kitchen\n               \n\n\n\n\n     ...   
3      B007Y1LJIU  Home & Kitchen\n               \n\n\n\n\n     ...   
4      B00DOOD7W2  Home & Kitchen\n               \n\n\n\n\n     ...   
...           ...                                                ...   
11691  B0DT9RFTWY  Health & Household\n               \n\n\n\n\n ...   
11692  B0DT9PRL11  Health & Household\n               \n\n\n\n\n ...   
11693  B0DT9RY8H9  Health & Household\n               \n\n\n\n\n ...   
11694  B0DT9RQHK1  Health & Household\n               \n\n\n\n\n ...   
11695  B0DT9TBVCP  Health & Household\n               \n\n\n\n\n ...   

                                   Cleaned_Bullet_Points  \
0      [WHITE SOPHISTICATION: Enhance your ev

In [17]:
validation_df.product_specifications[1572]

"Product Material: Stainless Steel                                 Product Color: Silver                                 Pack Size: 1 count                                 Product Dimensions: 5.8''L x 3.0''W x 3.8''H                                 Product SKU: RWT0189"

In [18]:
import pandas as pd

# Define the set of headers
headers_set = {
    'Access Method', 'Age Gender Category','Is the item dishwasher safe?','Is the item microwaveable?', 'Age Range Description','Product Care Instructions', 'Allergen Information', 'Amperage', 'Animal Theme', 
    'Annual Energy Consumption', 'Assembly Instructions', 'Astm Fluid Rating', 'Athlete', 'Auto Part Position', 'Back', 
    'Baking Mix Variety', 'Band', 'Base', 'Base Type', 'Batteries Included', 'Batteries Required', 'Battery', 'Blade', 
    'Bottle Type', 'Bottom', 'Bottom Diameter', 'Bowl', 'Bowl Capacity', 'Brand', 'Brightness', 'Brim Width', 'Bristle', 
    'Bulb', 'Bullet Point', 'Cable', 'Caliber', 'California Proposition 65', 'Candy Consistency', 'Cap', 'Capacity', 
    'Capacity Name', 'Care Instructions', 'Caster', 'Center To Center Spacing', 'Certification Number', 'Chain Description', 
    'Chamber', 'Chemical Names', 'Child Parent Sku Relationship', 'Closure', 'Closure Material', 'Coarseness Rating', 
    'Coating', 'Collar Style', 'Collection', 'Collection Description', 'Color', 'Color Rendering Index', 'Color Temperature', 
    'Compatibility Options', 'Compatible Devices', 'Compatible Material', 'Compliance Media', 'Computer Memory', 
    'Configuration', 'Connectivity Protocol', 'Connectivity Technology', 'Construction Type', 'Container', 
    'Contains Liquid Contents', 'Control Method', 'Control Type', 'Controller Type', 'Controls Type', 'Cooking Surface Length Width', 
    'Core Size', 'Corner Style', 'Cost Price', 'Country As Labeled', 'Country Of Origin', 'Coverage', 'Cpsia Cautionary Statement', 
    'Cuisine', 'Current Rating', 'Customer Package Type', 'Cutting Diameter', 'Cycle Options', 'Cylinder Axis', 
    'Cylinder Correction', 'Denomination', 'Department', 'Deprecated Map Price', 'Deprecated Offering Start Date', 
    'Deprecated Shaft Style Type', 'Design Name', 'Dial', 'Diet Type', 'Digital Storage Capacity', 'Display', 
    'Display Maximum Weight Recommendation', 'Door', 'Drinking Cup Form', 'Duration', 'Each Unit Count', 'Edition', 
    'Embellishment Feature', 'Energy Consumption', 'Energy Guide', 'Energy Star', 'Eu Energy Label Efficiency Class', 
    'Exterior Finish', 'External Product Id', 'Fabric Type', 'Fabric Weight', 'Fastener', 'Fc Shelf Life', 
    'Fcc Radio Frequency Emission Compliance', 'Fill Material', 'Filter Class', 'Finish Type', 'Fit Type', 'Flavor', 
    'Folded Item Length Width Height', 'Form Factor', 'Frame', 'Furniture Finish', 'Gauge', 'Gem Type', 'Generic Keyword', 
    'Generic Type', 'Ghs', 'Giftwrap Capacity Id', 'Giftwrap Style Id', 'Giftwrap Type Id', 'Golf Club Flex', 'Golf Club Loft', 
    'Graduation Interval', 'Graphics Description', 'Grip', 'Grit', 'Gtin Exemption Reason', 'Hand Orientation', 'Handle', 
    'Hard Disk', 'Has Automatic Shutoff', 'Has Lid', 'Has Night Vision', 'Has Nonstick Coating', 'Hat Form Type', 'Hazmat', 
    'Head', 'Head Width String', 'Headband Color', 'Headwear Size', 'Heating Element Type', 'Hem Stitch Type', 'Hose Length', 
    'Hub Height', 'Identity Package Type', 'Image Locator Egus', 'Image Locator Elfl', 'Immersion Depth', 'Import Designation', 
    'Included Components', 'Included Feature', 'Indoor Outdoor Usage', 'Ink', 'Inlet', 'Inner', 'Inscription', 'Inseam', 
    'Inside', 'Installation Type', 'Insulation', 'Interior Color', 'Is Assembly Required', 'Is Autographed', 'Is Cordless', 
    'Is Customizable', 'Is Dishwasher Safe', 'Is Electric', 'Is Expiration Dated Product', 'Is Fragile', 'Is Heat Sensitive', 
    'Is Liquid Double Sealed', 'Is Microwaveable', 'Is Oem Authorized', 'Is Oven Safe', 'Is Portable', 'Is Stain Resistant', 
    'Is Trade Item Orderable Unit', 'Is Waterproof', 'Item Booking Date', 'Item Depth', 'Item Depth Width Height', 
    'Item Diameter', 'Item Dimensions','Item Dimensions W x H', 'Item Dimensions Fraction', 'Item Display Dimensions', 'Item Display Weight', 
    'Item Firmness Description', 'Item Form', 'Item Length', 'Item Length Description', 'Item Length Width', 
    'Item Length Width Height', 'Item Length Width Thickness', 'Item Name', 'Item Package Dimensions', 'Item Package Weight', 
    'Item Shape', 'Item Thickness', 'Item Torque', 'Item Type Keyword', 'Item Type Name', 'Item Volume', 'Item Weight', 
    'Item Width Diameter Height', 'Item Width Height', 'Knife Form', 'Lamp Type', 'Language', 'League Name', 
    'Legal Compliance Certifications', 'Length Range', 'Lens', 'Lifecycle Supply Type', 'Lifestyle', 'Light Color', 
    'Light Path Distance', 'Light Source', 'Lighting Method', 'Limited Edition Range', 'Line Size', 'Line Weight', 
    'Lining Description', 'Liquid Contents Description', 'Liquid Packaging Type', 'Liquid Volume', 'List Price', 
    'Lithium Battery', 'Load Capacity', 'Lock Type', 'Lower Temperature Rating', 'Magnification Strength', 
    'Main Product Image Locator', 'Manufacture Year', 'Manufacturer', 'Manufacturer Maximum Age', 'Manufacturer Minimum Age', 
    'Master Pack Layers Per Pallet Quantity', 'Master Packs Per Layer Quanti', 'Master Packs Per Layer Quantity', 'Material', 
    'Material Composition', 'Material Feature', 'Material Type Free', 'Matte Style', 'Maximum Compatible Light Source Wattage', 
    'Maximum Cut Depth', 'Maximum Power', 'Maximum Range', 'Maximum Range Indoors', 'Maximum Range Open Space', 'Maximum Size', 
    'Maximum Temperature', 'Maximum Weight Recommendation', 'Measurement Accuracy', 'Measurement System', 'Measurement Type', 
    'Measuring Range', 'Media Type', 'Melting Temperature', 'Memory Storage Capacity', 'Merchant Suggested Asin', 'Metal Type', 
    'Minimum Required Door Width', 'Model Name', 'Model Number', 'Mounting Distance', 'Mounting Type', 'Multibox', 'Neck', 
    'Net Content Area', 'Net Content Count', 'Net Content Length', 'Net Content Volume', 'Net Content Weight', 'Night Vision Range', 
    'Noise Level', 'Num Batteries', 'Number Of Blades', 'Number Of Boxes', 'Number Of Compartments', 'Number Of Heating Elements', 
    'Number Of Hooks', 'Number Of Items', 'Number Of Labels', 'Number Of Levels', 'Number Of Light Sources', 'Number Of Lithium Ion Cells', 
    'Number Of Lithium Metal Cells', 'Number Of Options Cycles', 'Number Of Pieces', 'Number Of Pockets', 'Number Of Programs', 
    'Number Of Sets', 'Number Of Settings', 'Number Of Shelves', 'Number Of Speeds', 'Number Of Trays', 'Number Of Wheels', 'Numeric Theme', 
    'Nutritional Info', 'Occasion', 'Occasion Type', 'Oem Equivalent Part Number', 'Oil Capacity', 'Opening Mechanism', 
    'Opening Shape', 'Operating Frequency', 'Operating System', 'Operation Mode', 'Optical Power', 'Other Product Image Locator 1', 
    'Other Product Image Locator 2', 'Other Product Image Locator 3', 'Other Product Image Locator 4', 'Other Product Image Locator 5', 
    'Other Product Image Locator 6', 'Other Product Image Locator 7', 'Other Product Image Locator 8', 'Outer', 'Outlet', 'Outside Diameter', 
    'Package Contains Sku', 'Package Level', 'Package Type', 'Paper Finish', 'Paper Size', 'Paper Towel Fold Type', 'Parentage Level', 
    'Part Number', 'Pattern', 'Pattern Type', 'Pesticide Marking', 'Ph Range', 'Pile Height', 'Plant Or Animal Product Type', 
    'Platform For Display', 'Ply', 'Pocket Description', 'Pore Size', 'Power Source Type', 'Processor Description', 'Product Benefit', 
    'Product Category', 'Product Expiration Type', 'Product Name', 'Product Site Launch Date', 'Product Subcategory', 'Projection Distance', 
    'Range', 'Readout Accuracy', 'Recommended Uses For Product', 'Recycled Content Percentage', 'Region Of Origin', 'Registration Status', 
    'Regulatory Compliance Certification', 'Repairability Index', 'Required Product Compliance Certificate', 'Reservoir Capacity', 
    'Resolution', 'Response Time', 'Reusability', 'Rim', 'Ring', 'Roll', 'Roll Quantity', 'Room Type', 'Rotation Period', 'Rtip Directions', 
    'Rtip Earliest Shipping Date', 'Rtip Indications', 'Rtip Ingredients', 'Rtip Inner Packs Per Master Pack', 'Rtip Items Per Inner Pack', 
    'Rtip Order Aggregate Type', 'Rtip Product Description', 'Rtip Safety Warning', 'Rug Form Type', 'Safety Data Sheet Url', 'Scent', 
    'Seasons', 'Seating Capacity', 'Seating Group', 'Septa Type', 'Series Title', 'Service Level', 'Service Plan', 'Service Provider', 
    'Serving Recommendation', 'Shade', 'Shaft', 'Shank', 'Sheet Count', 'Sheet Count Per Unit', 'Shelf', 'Shelf Thickness', 'Shelf Type', 
    'Size', 'Size Per Pearl', 'Skin Type', 'Sleeve', 'Slot Count', 'Software Included', 'Special Feature', 'Special Ingredients', 
    'Special Size Type', 'Specialty', 'Specific Uses For Product', 'Specification Met', 'Sport Type', 'Spout Outside Diameter', 
    'Sterility Rating', 'Storage Volume', 'Street Date', 'Style', 'Subject Character', 'Subject Keyword', 'Sugar Candy Type', 
    'Sun Protection', 'Supplier Declared Dg Hz Regulation', 'Support Type', 'Supported Media Size', 'Surface Recommendation', 
    'Swatch Product Image Locator', 'Switch Type', 'Target Audience', 'Target Audience Keyword', 'Target Gender', 'Target Use Body Part', 
    'Team Name', 'Temperature Range', 'Temperature Rating', 'Temperature Rating Degrees', 'Tensile Strength', 'Tension Level', 'Test Type', 
    'Theme', 'Thread', 'Tire', 'Total Diamond Weight', 'Total Eaches', 'Total Gem Weight', 'Total Metal Weight', 'Towel Form Type', 
    'Trade Size Name', 'Tray Type', 'Unit Count', 'Upper Temperature Rating', 'Variation Theme', 'Vehicle Fitment', 'Version For Country', 
    'Viewing Angle', 'Voltage', 'Volume Capacity Name', 'Waist', 'Warranty Description', 'Warranty Type', 'Water Consumption', 
    'Water Resistance Level', 'Wattage', 'Weave Type', 'Website Shipping Weight', 'Weigh Scale Type', 'Weight Capacity', 'Weight Range', 
    'Wheel', 'Wide End Diameter', 'Width Range', 'Wireless Provider', 'Zoom Ratio', 'Product Dimensions', 'Item model number', 
    'Customer Reviews', 'Best Sellers Rank', 'Date First Available', 'Bowl Material', 'Shape', 'Theme', 'Product Care Instructions', 'ASIN', 
    'Is Discontinued By Manufacturer', 'Date First Available','Special Features'
}

# Function to process product information and extract headers and values
def process_product_information(row):
    if pd.notnull(row['product_information']):
        # Clean and split the product information
        product_info = row['product_information'].split('\n')
        
        # Remove unwanted sections and clean up the data
        cleaned_info = []
        for line in product_info:
            cleaned_line = line.strip()
            if cleaned_line:
                cleaned_info.append(cleaned_line)
        
        # Extract headers and values based on the provided set of headers
        headers = []
        values = []
        i = 0
        while i < len(cleaned_info):
            header = cleaned_info[i]
            if header in headers_set:
                headers.append(header)
                if i + 1 < len(cleaned_info):
                    value = cleaned_info[i + 1].strip()
                    if value not in headers_set:
                        values.append(value)
                        i += 2
                    else:
                        values.append('')
                        i += 1
                else:
                    values.append('')
                    i += 1
            else:
                i += 1
        
        # Add extracted headers and values to the row
        for header, value in zip(headers, values):
            row[header] = value

    return row

# Apply the processing function to the DataFrame
validation_df = validation_df.apply(process_product_information, axis=1)

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Display the processed DataFrame
validation_df.head

<bound method NDFrame.head of              ASIN Base Type                 Best Sellers Rank Bottle Type  \
0      B00DOOFTAU       NaN  #407,232 in Health & Household (         NaN   
1      B00NJSYCMM       NaN                               NaN         NaN   
2      B00DOOCR7I       NaN                               NaN         NaN   
3      B007Y1LJIU       NaN    #412,744 in Kitchen & Dining (         NaN   
4      B00DOOD7W2       NaN    #526,182 in Kitchen & Dining (         NaN   
...           ...       ...                               ...         ...   
11691  B0DT9RFTWY       NaN                               NaN         NaN   
11692  B0DT9PRL11       NaN                               NaN         NaN   
11693  B0DT9RY8H9       NaN                               NaN         NaN   
11694  B0DT9RQHK1       NaN                               NaN         NaN   
11695  B0DT9TBVCP       NaN                               NaN         NaN   

      Bottom Diameter Bowl Bowl Capacity Bowl

In [19]:
import pandas as pd
import re
# Define a function to extract the product type with a check for NaN or non-string values
def extract_product_type(text):
    if isinstance(text, str):  # Check if text is a string
        match = re.search(r"metricParameters\.productType = '(.*?)';", text)
        return match.group(1) if match else None
    return None  # Return None if the input is not a string

# Apply the function to the column and store the results in a new column
validation_df['product_type'] = validation_df['products_overview'].apply(extract_product_type)

validation_df.head

C:\Users\sudha\AppData\Local\Temp\ipykernel_34540\52616948.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  validation_df['product_type'] = validation_df['products_overview'].apply(extract_product_type)


<bound method NDFrame.head of              ASIN Base Type                 Best Sellers Rank Bottle Type  \
0      B00DOOFTAU       NaN  #407,232 in Health & Household (         NaN   
1      B00NJSYCMM       NaN                               NaN         NaN   
2      B00DOOCR7I       NaN                               NaN         NaN   
3      B007Y1LJIU       NaN    #412,744 in Kitchen & Dining (         NaN   
4      B00DOOD7W2       NaN    #526,182 in Kitchen & Dining (         NaN   
...           ...       ...                               ...         ...   
11691  B0DT9RFTWY       NaN                               NaN         NaN   
11692  B0DT9PRL11       NaN                               NaN         NaN   
11693  B0DT9RY8H9       NaN                               NaN         NaN   
11694  B0DT9RQHK1       NaN                               NaN         NaN   
11695  B0DT9TBVCP       NaN                               NaN         NaN   

      Bottom Diameter Bowl Bowl Capacity Bowl

In [20]:
import datetime

# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
file_path= f"webscrape_catalog as on {current_date}.xlsx"
validation_df.to_excel(file_path, index=False)
